In [58]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [131]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://13.57.193.25:9000/",        "nid": 3}}

env = connections[network]

In [290]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx6927160050f452ae32163c895c08e0171aa6073c'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx112c953f6f505f688401af0daf0345cb87ef5492'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxf90da35301d655e7fc760d5576b449a534f1987f'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx226fa714024eac7cf2e7f8fa0847f59d624b1a8f'}}

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [133]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [134]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [299]:
params = {"_base": "USD", "_quote": "ICX"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dummy_oracle']['SCORE'])\
                    .method("get_reference_data")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'rate': '0x2454d8dcb2cf44bb',
 'last_update_base': '0x5b131c980674d',
 'last_update_quote': '0x5b131c464af80'}

In [323]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
deploy = ['loans', 'staking', 'dex', 'sicx', 'icd', 'dummy_oracle']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'
contracts

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx6927160050f452ae32163c895c08e0171aa6073c'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx112c953f6f505f688401af0daf0345cb87ef5492'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxf90da35301d655e7fc760d5576b449a534f1987f'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx226fa714024eac7cf2e7f8fa0847f59d624b1a8f'}}

## Deploy All SCOREs

In [296]:
# Deploy a single SCORE
score = 'dummy_oracle'
loc = {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35'}
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(loc["zip"]))\
    .build()

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


In [264]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx6927160050f452ae32163c895c08e0171aa6073c'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx112c953f6f505f688401af0daf0345cb87ef5492'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxf90da35301d655e7fc760d5576b449a534f1987f'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35'}}

## Update a SCORE

In [324]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xed53f0b7147a14961511c3e6ca74a31ba433fee6d2daa33be3facbd19b6f85b3',
 'blockHeight': 2716,
 'blockHash': '0xed0c206ada565409c1c9db8d808cd3513a74fa77c69990a75ccd61c8094eb041',
 'txIndex': 0,
 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c',
 'scoreAddress': 'cx6927160050f452ae32163c895c08e0171aa6073c',
 'stepUsed': 434906000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 434906000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Set parameters

In [320]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'staking', 'method': 'set_sICX_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'set_sicx_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE']}},
            {'contract': 'loans', 'method': 'set_staking_contract', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'loans', 'method': 'set_oracle_address', 'params':{'_address': contracts['dummy_oracle']['SCORE']}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]

In [321]:
for sett in settings:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

{'txHash': '0x56795208fcfd2d0fe3f551ef7edfade2f1c0a77973fc89913ca46a2929f2b3d5', 'blockHeight': 2691, 'blockHash': '0x22a9c168898d2b0106794b5b8b3faaea9655086e8675b6990fa4a33b590812ce', 'txIndex': 0, 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'stepUsed': 1028450, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1028450, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'txHash': '0xd84ac048b51c0051c7413d24dd23fefc84a1a19a53394bef9308ee7898ad4c5c', 'blockHeight': 2692, 'blockHash': '0xaf7cdb442a9c929689b4404910e9b6f4fa1ae25d9a94b2cbfcdcafe72b24903f', 'txIndex': 0, 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'stepUsed': 1028050, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1028050, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xf461e3bf06dc6e47897268e252fa1df29fa1721965f65b1ec00993b0e394ac00', 'blockHeight': 2693, 'blockHash': '0x96790bc9acbdb891ca5ffb635646ead38a47a21b2ab5b4da5b0bfd649972ea7c', 'txIndex': 0, 'to': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'stepUsed': 1018050, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1018050, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Run through read-only methods on all SCOREs

In [302]:
ro_methods = {"loans": ["name", "get_oracle_address", "get_available_assets"],
              "staking": ["name", "get_sICX_address", "get_rate"],
              "dex": ["name", "get_exchange_on"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [326]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
name: BalancedLoans
get_oracle_address: cx226fa714024eac7cf2e7f8fa0847f59d624b1a8f
get_available_assets: [['ICD', 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35', 'USD']]

SCORE: staking
name: Staking
get_sICX_address: cxf90da35301d655e7fc760d5576b449a534f1987f
get_rate: 0xde0b6b3a7640000

SCORE: dex
name: BalancedDEX
get_exchange_on: 0x0

SCORE: sicx
name: StakedICX
symbol: sICX
decimals: 0x12
totalSupply: 0x152d02c7e14af6800000
getAdmin: cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7

SCORE: icd
name: ICONDollar
symbol: ICD
decimals: 0x12
totalSupply: 0x15af1d78b58c400000
getAdmin: cx6927160050f452ae32163c895c08e0171aa6073c


## Test a single method

In [316]:
params = {'_symbol': 'USD'}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_price_in_icx')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

Raised JSONRPCException while returning the custom response. Error message: {'code': -32009, 'message': 'No permission to write'}


JSONRPCException: {'code': -32009, 'message': 'No permission to write'} (Json rpc error)

In [143]:
wallet.get_address()

'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'

## Depost ICX and Mint ICD Loan

In [325]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_for\": \"hxe7af5fcfd8dfc67530a01a0e403882687528dfcb\", \"_asset\": \"ICD\", \"_amount\": 400000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(100000 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x11912d06de07b98dc19b8e756f9f9bdc1e7edda8dd88cf875aa244f876362337', 'blockHeight': 2720, 'blockHash': '0xbe916e8aad54d857dcc7f06aa4d41ff091864bd59e55cca5e16c2cdd7e02386e', 'txIndex': 0, 'to': 'cx4c94c5802cd7c4c9b9e1a5eef96c71cbb024f8f7', 'stepUsed': 1242700, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1242700, 'eventLogs': [{'scoreAddress': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x15af1d78b58c400000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cxcdc859b49084b4bfe63d8efb711f71bdc1482d35', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x15af1d78b58c400000', '0x4e6f6e65']}, {'scoreAddress': 'cx6927160050f452ae32163c895c08e0171aa6073c', 'indexed': ['OriginateLoan(Address,str,int,str